In [1]:
# This notebook is to be used on pre-processed fMRI timeseries data 

In [1]:
import os

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


import nibabel as nib
from nilearn.input_data import NiftiMasker

from sklearn.preprocessing import StandardScaler
from sklearn.metrics.pairwise import cosine_similarity

import graph_tool.all as gt

import csv
import pickle

In [2]:
# Load an image from a single brain:
def img_load(folder, img):
    img_path = os.path.join(folder, img)
    fmri_image = nib.load(img_path)
    #print(fmri_image.shape)
    return fmri_image

In [3]:
# Load a resampled, binarised GM mask
def mask_img_load(folder, img):
    img_path = os.path.join(folder, img)
    mask = nib.load(img_path)
    return mask

In [4]:
# Extract the timeseries from a single session
def extract_time_series(fmri_image, mask):
    brain_masker = NiftiMasker(mask, memory='nilearn_cache', memory_level=1, verbose=0)
    brain_time_series = brain_masker.fit_transform(fmri_image)
    return brain_time_series, brain_masker

In [5]:
# Compute cosine similarity (N.B. same as Pearson correlation when data is centered)
def cos_sim_func(time_series):
    cos_sim = cosine_similarity(time_series.T, time_series.T)
    return cos_sim

In [6]:
# Threshold the adjacency matrix
def thresh_mat(adjacency_matrix):
    # threshold at r > 0.25
    adjacency_matrix[adjacency_matrix < 0.25] = 0
    # fill diagonal with zeroes
    np.fill_diagonal(adjacency_matrix, 0)
    return adjacency_matrix

In [7]:
# https://carlonicolini.github.io/sections/science/2018/09/12/weighted-graph-from-adjacency-matrix-in-graph-tool.html
# To create an undirected, weighted graph with graph-tool
def to_graph_tool(adj):
    g = gt.Graph(directed=False)
    edge_weights = g.new_edge_property('double')
    g.edge_properties['weight'] = edge_weights
    # Set the lower triangle of the adjacency matrix and the diagonal to 0
    nnz = np.nonzero(np.triu(adj,1))
    # Get the number of edges (i.e. non-zero values)
    nedges = len(nnz[0])
    # Create the edge value list
    g.add_edge_list(
        # Create rows of THREE values
        np.hstack(
        [
        # Transpose nnz so that you have TWO values in each row, where
        # the first is the row index and the second is the column index
        # of this particular edge
        np.transpose(nnz),
        # Get the 3RD values for each row, i.e. the edge weight
        np.reshape(adj[nnz],(nedges,1))
        ]
        ),
    # If given, eprops should specify an iterable containing edge property maps that will be filled with the remaining values at each row, if there are more than two.
    eprops=[edge_weights])
    return g

In [8]:
# Get n of degrees for each voxel
def calc_dc(adjacency_matrix):
    g = to_graph_tool(adjacency_matrix)
    dc = g.get_total_degrees([i for i in range(adjacency_matrix.shape[0])])
    return dc

In [9]:
# Z-score
def z_score_dc(dc_array):
    dc = dc_array.reshape(-1, 1)
    scaler = StandardScaler()
    dc_scaled = scaler.fit_transform(dc)
    return dc_scaled

In [10]:
# Transform the FC or DC array back into a NIfTI volume and save
def array_to_nifti(brain_masker,array):
    img = brain_masker.inverse_transform(array.T)
    return img

In [11]:
###################################### Run the DC analysis on the whole dataset ######################################

In [12]:
# To calculate functional connectivity and degree centrality from an adjacency matrix
def calc_dc_auto(fMRIroot, fMRI_txt, MASKroot, GM_mask, ADDitional_mask):
    
    # transform the txt file into a Python list... of lists!
    fmri_list = []
    with open(fMRI_txt, newline='') as inputfile:
        for row in csv.reader(inputfile):
            fmri_list.append(row)
    
    for i in range(len(fmri_list)):
        # load an image from a single brain:
        fmri_image = img_load(fMRIroot, fmri_list[i][0])
        print("Participant " + fmri_list[i][0])
        print("The image dimensions are: " + str(fmri_image.shape))
        
        # load the GM mask
        gm_mask = mask_img_load(MASKroot, GM_mask)
        print("GM mask loaded.")
        
        # create a NiftiMasker object and extract the time series
        brain_time_series, brain_masker = extract_time_series(fmri_image, gm_mask)
        print("Time series extracted.")
        
        # calculate the correlations between each pair of voxels
        cos_sim = cos_sim_func(brain_time_series)
        print("Correlations calculated.")
        
        # threshold the matrix
        adjacency_matrix = thresh_mat(cos_sim)
        print("Adjacency matrix thresholded.")
        
        # save the matrix
        #matrix_name = fmri_list[i][0][:12] + "_adj_matrix"
        #cm = open(matrix_name + ".pkl.gz", 'wb')
        #pickle.dump(adjacency_matrix, cm)
        #cm.close
        #print("Adjacency matrix saved.")
        
        # calculate the degree centrality (DC)
        dc = calc_dc(adjacency_matrix)
        print("Raw DC values calcualted.")
        
        # convert the raw DC matrix and save it as a NIfTI image
        dc_img = array_to_nifti(brain_masker,dc)
        dc_img_name = fmri_list[i][0][:12] + "_DC_raw"
        nib.save(dc_img, dc_img_name)
        print("Raw DC image of the whole GM of " + fmri_list[i][0][:12] + " saved.")
        
        # load (an) intersection/ROI mask(s)
        for k in range(len(ADDitional_mask)):
            
            # load the additional mask
            additional_mask = mask_img_load(MASKroot, ADDitional_mask[k])
            mask_name = ADDitional_mask[k].split('.')[0] 
            print("Additional mask " + mask_name + " loaded.")
            
            # extract masked DC matrix
            masked_dc_matrix, masked_brain_masker = extract_time_series(dc_img, additional_mask)
            print("Masked DC matrix extracted.") # Here is the distribution. Please check whether there are too many zero values. If there are, Z-scoring should be avoided..")
            #plt.hist(masked_dc_matrix
                    #,bins = 20)
            #hist_name = fmri_list[i][0][:12] + "_" + mask_name + "_hist"
            #plt.savefig(hist_name, format=png)
            
            # z-score the DC matrix
            dc_scaled = z_score_dc(masked_dc_matrix)
            print("DC matrix z-scored.")

            # save z-scored DC matrix as a NIfTI file
            # get the bit (of the mask name) before the first '.' to ensure it's not including file extensions  
            dc_img_z = array_to_nifti(masked_brain_masker,dc_scaled)
            dc_img_z_name = fmri_list[i][0][:12] + "_" + mask_name + "_DC_z"
            nib.save(dc_img_z, dc_img_z_name)
            print("Z-scored DC image of " + fmri_list[i][0][:12] + " masked with the " + mask_name + " mask saved.")

    print("Done!")

In [13]:
# Run the analysis
calc_dc_auto("/Volumes/Seagate Dr/PhD/Tianeptine/BOLD_data",
             "/Users/mishodimitrov/Downloads/PhD/Analysis/Tianeptine/Data/trada_session_list.txt",
             "/Users/mishodimitrov/Downloads/PhD/Analysis/Tianeptine/Data/Masks",
            "final_resampled_gm.nii.gz",
            ["asd_td.nii.gz", "td_asd.nii.gz"
             #, "tia_int_mask_0.6.nii"
            ])

Participant BRCTRADA001B.nii.gz
The image dimensions are: (55, 65, 55, 205)
GM mask loaded.
Time series extracted.
Correlations calculated.
Adjacency matrix thresholded.
Raw DC values calcualted.
Raw DC image of the whole GM of BRCTRADA001B saved.
Additional mask asd_td loaded.
Masked DC matrix extracted.
DC matrix z-scored.
Z-scored DC image of BRCTRADA001B masked with the asd_td mask saved.
Additional mask td_asd loaded.
Masked DC matrix extracted.
DC matrix z-scored.
Z-scored DC image of BRCTRADA001B masked with the td_asd mask saved.
Participant BRCTRADA001D.nii.gz
The image dimensions are: (55, 65, 55, 205)
GM mask loaded.
Time series extracted.
Correlations calculated.
Adjacency matrix thresholded.
Raw DC values calcualted.
Raw DC image of the whole GM of BRCTRADA001D saved.
Additional mask asd_td loaded.
Masked DC matrix extracted.
DC matrix z-scored.
Z-scored DC image of BRCTRADA001D masked with the asd_td mask saved.
Additional mask td_asd loaded.
Masked DC matrix extracted.
D

/opt/anaconda3/envs/Project_3_TIA/lib/python3.9/site-packages/nilearn/input_data/nifti_masker.py:398: UserWarning: Persisting input arguments took 0.64s to run.
If this happens often in your code, it can cause performance problems 
(results will be correct in all cases). 
The reason for this is probably some large input arguments for a wrapped
 function (e.g. large strings).
THIS IS A JOBLIB ISSUE. If you can, kindly provide the joblib's team with an
 example so that they can fix the problem.
  data = self._cache(filter_and_mask,


Time series extracted.
Correlations calculated.
Adjacency matrix thresholded.
Raw DC values calcualted.
Raw DC image of the whole GM of BRCTRADA005C saved.
Additional mask asd_td loaded.
Masked DC matrix extracted.
DC matrix z-scored.
Z-scored DC image of BRCTRADA005C masked with the asd_td mask saved.
Additional mask td_asd loaded.
Masked DC matrix extracted.
DC matrix z-scored.
Z-scored DC image of BRCTRADA005C masked with the td_asd mask saved.
Participant BRCTRADA006B.nii.gz
The image dimensions are: (55, 65, 55, 205)
GM mask loaded.
Time series extracted.
Correlations calculated.
Adjacency matrix thresholded.
Raw DC values calcualted.
Raw DC image of the whole GM of BRCTRADA006B saved.
Additional mask asd_td loaded.
Masked DC matrix extracted.
DC matrix z-scored.
Z-scored DC image of BRCTRADA006B masked with the asd_td mask saved.
Additional mask td_asd loaded.
Masked DC matrix extracted.
DC matrix z-scored.
Z-scored DC image of BRCTRADA006B masked with the td_asd mask saved.
Part

Adjacency matrix thresholded.
Raw DC values calcualted.
Raw DC image of the whole GM of BRCTRADA013C saved.
Additional mask asd_td loaded.
Masked DC matrix extracted.
DC matrix z-scored.
Z-scored DC image of BRCTRADA013C masked with the asd_td mask saved.
Additional mask td_asd loaded.
Masked DC matrix extracted.
DC matrix z-scored.
Z-scored DC image of BRCTRADA013C masked with the td_asd mask saved.
Participant BRCTRADA013D.nii.gz
The image dimensions are: (55, 65, 55, 205)
GM mask loaded.
Time series extracted.
Correlations calculated.
Adjacency matrix thresholded.
Raw DC values calcualted.
Raw DC image of the whole GM of BRCTRADA013D saved.
Additional mask asd_td loaded.
Masked DC matrix extracted.
DC matrix z-scored.
Z-scored DC image of BRCTRADA013D masked with the asd_td mask saved.
Additional mask td_asd loaded.
Masked DC matrix extracted.
DC matrix z-scored.
Z-scored DC image of BRCTRADA013D masked with the td_asd mask saved.
Participant BRCTRADA014B.nii.gz
The image dimensions

/opt/anaconda3/envs/Project_3_TIA/lib/python3.9/site-packages/nilearn/input_data/nifti_masker.py:398: UserWarning: Persisting input arguments took 0.50s to run.
If this happens often in your code, it can cause performance problems 
(results will be correct in all cases). 
The reason for this is probably some large input arguments for a wrapped
 function (e.g. large strings).
THIS IS A JOBLIB ISSUE. If you can, kindly provide the joblib's team with an
 example so that they can fix the problem.
  data = self._cache(filter_and_mask,


...

/opt/anaconda3/envs/Project_3_TIA/lib/python3.9/site-packages/nilearn/input_data/nifti_masker.py:398: UserWarning: Persisting input arguments took 0.50s to run.
If this happens often in your code, it can cause performance problems 
(results will be correct in all cases). 
The reason for this is probably some large input arguments for a wrapped
 function (e.g. large strings).
THIS IS A JOBLIB ISSUE. If you can, kindly provide the joblib's team with an
 example so that they can fix the problem.
  data = self._cache(filter_and_mask,


Time series extracted.
Correlations calculated.
Adjacency matrix thresholded.
Raw DC values calcualted.
Raw DC image of the whole GM of BRCTRADA107C saved.
Additional mask asd_td loaded.
Masked DC matrix extracted.
DC matrix z-scored.
Z-scored DC image of BRCTRADA107C masked with the asd_td mask saved.
Additional mask td_asd loaded.
Masked DC matrix extracted.
DC matrix z-scored.
Z-scored DC image of BRCTRADA107C masked with the td_asd mask saved.
Participant BRCTRADA107D.nii.gz
The image dimensions are: (55, 65, 55, 205)
GM mask loaded.
Time series extracted.
Correlations calculated.
Adjacency matrix thresholded.
Raw DC values calcualted.
Raw DC image of the whole GM of BRCTRADA107D saved.
Additional mask asd_td loaded.
Masked DC matrix extracted.
DC matrix z-scored.
Z-scored DC image of BRCTRADA107D masked with the asd_td mask saved.
Additional mask td_asd loaded.
Masked DC matrix extracted.
DC matrix z-scored.
Z-scored DC image of BRCTRADA107D masked with the td_asd mask saved.
Part

/opt/anaconda3/envs/Project_3_TIA/lib/python3.9/site-packages/nilearn/input_data/nifti_masker.py:398: UserWarning: Persisting input arguments took 0.50s to run.
If this happens often in your code, it can cause performance problems 
(results will be correct in all cases). 
The reason for this is probably some large input arguments for a wrapped
 function (e.g. large strings).
THIS IS A JOBLIB ISSUE. If you can, kindly provide the joblib's team with an
 example so that they can fix the problem.
  data = self._cache(filter_and_mask,


Time series extracted.
Correlations calculated.
Adjacency matrix thresholded.
Raw DC values calcualted.
Raw DC image of the whole GM of BRCTRADA111D saved.
Additional mask asd_td loaded.
Masked DC matrix extracted.
DC matrix z-scored.
Z-scored DC image of BRCTRADA111D masked with the asd_td mask saved.
Additional mask td_asd loaded.
Masked DC matrix extracted.
DC matrix z-scored.
Z-scored DC image of BRCTRADA111D masked with the td_asd mask saved.
Participant BRCTRADA112B.nii.gz
The image dimensions are: (55, 65, 55, 205)
GM mask loaded.
Time series extracted.
Correlations calculated.
Adjacency matrix thresholded.
Raw DC values calcualted.
Raw DC image of the whole GM of BRCTRADA112B saved.
Additional mask asd_td loaded.
Masked DC matrix extracted.
DC matrix z-scored.
Z-scored DC image of BRCTRADA112B masked with the asd_td mask saved.
Additional mask td_asd loaded.
Masked DC matrix extracted.
DC matrix z-scored.
Z-scored DC image of BRCTRADA112B masked with the td_asd mask saved.
Part

Adjacency matrix thresholded.
Raw DC values calcualted.
Raw DC image of the whole GM of BRCTRADA120B saved.
Additional mask asd_td loaded.
Masked DC matrix extracted.
DC matrix z-scored.
Z-scored DC image of BRCTRADA120B masked with the asd_td mask saved.
Additional mask td_asd loaded.
Masked DC matrix extracted.
DC matrix z-scored.
Z-scored DC image of BRCTRADA120B masked with the td_asd mask saved.
Participant BRCTRADA120C.nii.gz
The image dimensions are: (55, 65, 55, 205)
GM mask loaded.
Time series extracted.
Correlations calculated.
Adjacency matrix thresholded.
Raw DC values calcualted.
Raw DC image of the whole GM of BRCTRADA120C saved.
Additional mask asd_td loaded.
Masked DC matrix extracted.
DC matrix z-scored.
Z-scored DC image of BRCTRADA120C masked with the asd_td mask saved.
Additional mask td_asd loaded.
Masked DC matrix extracted.
DC matrix z-scored.
Z-scored DC image of BRCTRADA120C masked with the td_asd mask saved.
Participant BRCTRADA123B.nii.gz
The image dimensions